In [1]:
import torch
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob
import numpy as np
from sklearn.metrics import mean_squared_error

print(torch.__version__)

1.10.0+cu111


In [2]:
!pip3 install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
     |████████████████████████████████| 68.5 MB 144 kB/s 
     |████████████████████████████████| 190 kB 5.3 MB/s 


In [3]:
!git clone https://github.com/open-mmlab/mmflow.git

Cloning into 'mmflow'...
remote: Enumerating objects: 1089, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 1089 (delta 149), reused 124 (delta 83), pack-reused 836
Receiving objects: 100% (1089/1089), 21.18 MiB | 27.35 MiB/s, done.
Resolving deltas: 100% (522/522), done.


In [4]:
%cd /content/mmflow

/content/mmflow


In [5]:
!pip install -r requirements/build.txt

In [6]:
!pip install -v -e .

Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local
sysconfig: /usr
Additional context:
user = False
home

In [9]:
def mse(imageA, imageB):
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	return err

In [8]:
from mmflow.apis import inference_model, init_model
from mmflow.datasets import visualize_flow, write_flow
import mmcv

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
img = cv2.imread("/content/gdrive/MyDrive/Colab Notebooks/data/0000000116.png")
height, width, ch = img.shape


In [50]:
%%time

def inference(noise_video_name, config_file, checkpoint_file, size_1, size_2, size_3, size_4):

  images = glob.glob("/content/gdrive/MyDrive/Colab Notebooks/data/*")

  mse_results = []

  video = cv2.VideoWriter('/content/gdrive/MyDrive/Colab Notebooks/' + str(noise_video_name), cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width, height))


  model = init_model(config_file, checkpoint_file, device='cuda:0')

  for i in range(len(images)-1):

    noise = np.zeros([height, width, 3],dtype=np.uint8)
    noise.fill(1)
    # noise[150:250, 600:700].fill(0)
    noise[size_1:size_2, size_3:size_4].fill(0)

    img1 = cv2.imread(images[i])
    img2 = cv2.imread(images[i+1])
    result = inference_model(model, img1, img2)


    result_noise = inference_model(model, img1*noise, img2*noise)
    
    flow_map = visualize_flow(result_noise)
    video.write(flow_map)


    noise = np.zeros([height, width, 2],dtype=np.uint8)
    noise.fill(1)
    # noise[150:250, 600:700].fill(0)
    noise[size_1:size_2, size_3:size_4].fill(0)
    
    mse_results.append(mse(result*noise, result_noise*noise))

  video.release()
  cv2.destroyAllWindows()

  return mse_results

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [ ]:
# PWC

In [51]:
%%time

pwc_net_mse_50_50_mse = inference("pwcnet_50_50.avi", "configs/pwcnet/pwcnet_ft_4x1_300k_sintel_final_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/pwcnet_ft_4x1_300k_sintel_final_384x768.pth", 150, 200, 600, 650)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/pwcnet_ft_4x1_300k_sintel_final_384x768.pth
CPU times: user 1min 58s, sys: 1.46 s, total: 2min
Wall time: 1min 59s


In [52]:
np.mean(pwc_net_mse_50_50_mse)

0.3363088744253412

In [55]:
%%time

pwc_net_mse_100_100_mse = inference("pwcnet_100_100.avi", "configs/pwcnet/pwcnet_ft_4x1_300k_sintel_final_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/pwcnet_ft_4x1_300k_sintel_final_384x768.pth", 150, 250, 600, 700)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/pwcnet_ft_4x1_300k_sintel_final_384x768.pth
CPU times: user 2min, sys: 1.96 s, total: 2min 2s
Wall time: 2min 2s


In [56]:
np.mean(pwc_net_mse_100_100_mse)

1.1865125182102654

In [57]:
%%time

pwc_net_mse_150_150_mse = inference("pwcnet_150_150.avi", "configs/pwcnet/pwcnet_ft_4x1_300k_sintel_final_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/pwcnet_ft_4x1_300k_sintel_final_384x768.pth", 150, 300, 600, 750)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/pwcnet_ft_4x1_300k_sintel_final_384x768.pth
CPU times: user 2min, sys: 1.97 s, total: 2min 2s
Wall time: 2min 1s


In [58]:
np.mean(pwc_net_mse_150_150_mse)

8.767669348983095

In [59]:
%%time

pwc_net_mse_200_200_mse = inference("pwcnet_200_200.avi", "configs/pwcnet/pwcnet_ft_4x1_300k_sintel_final_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/pwcnet_ft_4x1_300k_sintel_final_384x768.pth", 150, 350, 600, 800)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/pwcnet_ft_4x1_300k_sintel_final_384x768.pth
CPU times: user 2min, sys: 8.51 s, total: 2min 8s
Wall time: 2min 7s


In [60]:
np.mean(pwc_net_mse_200_200_mse)

19.185147435329693

In [61]:
# FlowNet

In [62]:
%%time

flownet_mse_50_50_mse = inference("flownet_50_50.avi", "/content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth", 150, 200, 600, 650)

2022-01-29 12:23:04,736 - mmflow - INFO - Freeze the parameters in FlowNetCSS
2022-01-29 12:23:06,288 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


CPU times: user 4min 47s, sys: 2.99 s, total: 4min 50s
Wall time: 4min 51s


In [63]:
np.mean(flownet_mse_50_50_mse)

0.17451362658378622

In [64]:
flownet_mse_100_100_mse = inference("flownet_100_100.avi", "/content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth", 150, 250, 600, 700)

2022-01-29 12:27:56,438 - mmflow - INFO - Freeze the parameters in FlowNetCSS
2022-01-29 12:27:58,012 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


In [65]:
np.mean(flownet_mse_100_100_mse)

0.606409767537937

In [66]:
%%time

flownet_mse_150_150_mse = inference("flownet_150_150.avi", "/content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth", 150, 300, 600, 750)

2022-01-29 12:32:47,220 - mmflow - INFO - Freeze the parameters in FlowNetCSS
2022-01-29 12:32:48,803 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


CPU times: user 4min 47s, sys: 3.18 s, total: 4min 50s
Wall time: 4min 51s


In [67]:
np.mean(flownet_mse_150_150_mse)

2.00606850764791

In [68]:
flownet_mse_200_200_mse = inference("flownet_200_200.avi", "/content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth", 150, 350, 600, 800)

2022-01-29 12:37:38,290 - mmflow - INFO - Freeze the parameters in FlowNetCSS
2022-01-29 12:37:39,967 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/flownet/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


In [69]:
np.mean(flownet_mse_200_200_mse)

6.653170405819555

In [70]:
# raft

In [71]:
raft_mse_50_50_mse = inference("raft_50_50.avi", "/content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.py", "/content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.pth", 150, 200, 600, 650)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.pth


In [72]:
np.mean(raft_mse_50_50_mse)

5.14176043783015

In [73]:
raft_mse_100_100_mse = inference("raft_100_100.avi", "/content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.py", "/content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.pth", 150, 250, 600, 700)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.pth


In [74]:
np.mean(raft_mse_100_100_mse)

2.4569459428072697

In [75]:
raft_mse_150_150_mse = inference("raft_150_150.avi", "/content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.py", "/content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.pth", 150, 300, 600, 750)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.pth


In [76]:
np.mean(raft_mse_150_150_mse)

9.00192474941224

In [77]:
raft_mse_200_200_mse = inference("raft_200_200.avi", "/content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.py", "/content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.pth", 150, 350, 600, 800)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/raft/raft_8x2_100k_mixed_368x768.pth


In [78]:
np.mean(raft_mse_200_200_mse)

15.201781210852623

In [79]:
# lightflownet2

In [83]:
light_flownet_mse_50_50_mse = inference("light_flownet_50_50.avi", "/content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.pth", 150, 200, 600, 650)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.pth


In [84]:
np.mean(light_flownet_mse_50_50_mse)

0.2751221393011851

In [85]:
light_flownet_mse_100_100_mse = inference("light_flownet_100_100.avi", "/content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.pth", 150, 250, 600, 700)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.pth


In [86]:
np.mean(light_flownet_mse_100_100_mse)

1.0639372039002426

In [87]:
light_flownet_mse_150_150_mse = inference("light_flownet_150_150.avi", "/content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.pth", 150, 300, 600, 750)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.pth


In [88]:
np.mean(light_flownet_mse_150_150_mse)

5.330574969419675

In [89]:
light_flownet_mse_200_200_mse = inference("light_flownet_200_200.avi", "/content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.py", "/content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.pth", 150, 350, 600, 800)

load checkpoint from local path: /content/gdrive/MyDrive/Colab Notebooks/lightflownet2/liteflownet2_8x1_500k_flyingthing3d_subset_384x768.pth


In [90]:
np.mean(light_flownet_mse_200_200_mse)

13.639062030561481